In [14]:
# 操作 browser 的 API
# 強制等待 (執行期間休息一下)
# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 正規表達
import re

# 子處理程序，用來取代 os.system 的功能
import subprocess
from time import sleep

import ipywidgets as widgets
import requests
from bs4 import BeautifulSoup
from IPython.display import clear_output, display
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

In [15]:
def get_searchreview(driver, inputdata):
    # 打開 Google 網頁
    driver.get("https://www.google.com/maps")
    # 查找搜索框元素
    search_box = driver.find_element(By.ID, "searchboxinput")

    # 在搜索框中輸入關鍵字並按下回車鍵
    search_box.send_keys(inputdata)
    search_box.send_keys(Keys.ENTER)
    try:
        # 確保列表出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde.ecceSd")
            )
        )
    except Exception as e:
        print("沒找到任何結果", e)
        return false
    return True

In [16]:
# 創建一個下拉選單
dropdown = widgets.Dropdown(
    options=[
        ""  ,#search_food_type
    ],
    value="",
    description="choose:",
    disabled=False,
)

In [17]:
def TryGetRestaurantList(driver):
    restaurants = driver.find_elements(By.CSS_SELECTOR, ".hfpxzc")
    previous_name = None

    data = {"restaurants": []}

    for restaurant in restaurants:
        # 點擊清單列表
        restaurant.click()
        sleep(3)

        try:
            # 確保新餐廳名稱已經加載
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h1.DUwDvf.lfPIob"))
            )
        except Exception as e:
            print("沒找到餐廳的資料", e)
            continue  # 跳過此餐廳，繼續下一個

        # 獲取餐廳資訊
        restaurant_name = driver.find_element(By.CSS_SELECTOR, "h1.DUwDvf.lfPIob").text
        # aria_label = restaurant_name.get_attribute('aria-label')
        print(f"餐廳名稱: {restaurant_name}")

        if restaurant_name == previous_name:
            # print("重複的餐廳資料，跳過...")
            continue

        previous_name = restaurant_name

        # 餐廳類型
        restaurant_typelist = driver.find_elements(By.CSS_SELECTOR, "button.DkEaL")
        restaurant_type = "null"

        if len(restaurant_typelist) > 0:
            restaurant_type = restaurant_typelist[0].text
        print(f"餐廳類型 : {restaurant_type}")

        # avg pay
        restaurant_avg_pay = "null"
        restaurant_avg_paylist = driver.find_elements(
            By.XPATH,
            '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/span/span/span/span[2]/span/span',
        )
        if len(restaurant_avg_paylist) > 0:
            restaurant_avg_pay = restaurant_avg_paylist[0].get_attribute("aria-label")
        print(f"餐廳均消 : {restaurant_avg_pay}")

        restaurant_address = driver.find_element(
            By.CSS_SELECTOR, "div.Io6YTe.fontBodyMedium.kR99db"
        ).text
        print(f"餐廳地址 : {restaurant_address}")

        restaurant_business_hours = TryFindElements(driver, "div.t39EBf.GUrTXd")
        if len(restaurant_business_hours) > 0:
            business_hours_aria_label = restaurant_business_hours[0].get_attribute(
                "aria-label"
            )
            days_info = re.findall(
                r"(星期[日一二三四五六])、(\d{2}:\d{2} 到 \d{2}:\d{2})",
                business_hours_aria_label,
            )
            hours_dict = {day: hours for day, hours in days_info}
            print(f"餐廳營業時間資訊: {business_hours_aria_label}")

        # 餐廳評分
        restaurant_rating = "null"
        restaurant_ratinglist = driver.find_elements(
            By.XPATH,
            '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[1]/span[1]',
        )
        if len(restaurant_ratinglist) > 0:
            restaurant_rating = restaurant_ratinglist[0].text
        print(f"餐廳評分 : {restaurant_rating}")

        LatitudeAndLongitudelist = TryGetLatitudeAndLongitude(driver, restaurant_name)
        Latitude = "null"
        Longitude = "null"
        if LatitudeAndLongitudelist != None:
            Latitude = LatitudeAndLongitudelist[0]
            Longitude = LatitudeAndLongitudelist[1]

        # 放資料到 list 中
        data["restaurants"].append(
            {
                # "id": 1,
                "name": restaurant_name,
                "type": restaurant_type,
                "rating": restaurant_rating,
                "avg_pay": restaurant_avg_pay,
                "address": restaurant_address,
                "businessHours": hours_dict,
                "longitude": Longitude,
                "latitude": Latitude,
            }
        )

    return data

In [18]:
def TryFindElements(driver, element):
    elements = driver.find_elements(By.CSS_SELECTOR, element)
    return elements

In [19]:
from urllib.parse import quote

def TryGetLatitudeAndLongitude(driver, resaurant_name):

    url = driver.current_url
    # !3d25.0027336!4d121.3061443!
    regex_map_Latitude = r'(?<=!3d)(.+)(?=!4d)'
    regex_map_Longitude = r'(?<=!4d)(-?\d+\.\d+)'
    match_Latitude = re.search(regex_map_Latitude, url)
    match_Longitude = re.search(regex_map_Longitude, url)
    match_list = []
    
    if match_Latitude and match_Longitude:
            print(f'Latitude : {match_Latitude.group(0)}')
            print(f'Longitude : {match_Longitude.group(0)}')
            match_list.append(f'{match_Latitude.group(0)}')
            match_list.append(f'{match_Longitude.group(0)}')
    else:
        print(f'{resaurant_name}經緯度資訊沒取得，請確認')

    return match_list
    

In [20]:
# 等待圖片元素載入並可見
def TryWaitNewElementWait(driver):
    try:
        wait = WebDriverWait(driver, 10)
        wait.until(
            EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.Nv2PK"))
        )
    except TimeoutException:
        print("elements not loaded in time.")
        return False

    return True

In [21]:
def scroll(driver, elementlimit):
    element_count = 0

    while element_count < elementlimit:
        try:
            # 獲取所有可見元素
            elements = driver.find_elements(
                By.XPATH,
                '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[*]/div/a',
            )

            for element in elements:
                if WebDriverWait(driver, 10).until(EC.visibility_of(element)):
                    element_count += 1

            # 滾動到頁面底部
            scrollable_element = driver.find_element(
                By.XPATH,
                '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]',
            )
            scrollable_element.send_keys(Keys.END)
            sleep(2)  # 等待內容載入
        except Exception as e:
            print(e)
            break

    print(f"總共找到的元素數量: {element_count}")

In [22]:
# 將 list 存成 json
def saveJson(folderPath, listData, foodType):
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)

    with open(
        f"{folderPath}/restaurant_{foodType}.json", "w", encoding="utf-8"
    ) as file:
        file.write(json.dumps(listData, ensure_ascii=False, indent=4))

In [23]:
def start_driver():
    # 啟動瀏覽器工具的選項
    my_options = webdriver.ChromeOptions()
    # my_options.add_argument("--headless")              #不開啟實體瀏覽器背景執行
    my_options.add_argument("--start-maximized")         # 最大化視窗
    my_options.add_argument("--incognito")               # 開啟無痕模式
    my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
    my_options.add_argument("--disable-notifications")   # 取消 chrome 推播通知
    my_options.add_argument("--lang=zh-TW")              # 設定為正體中文
    return webdriver.Chrome(options=my_options)


button = widgets.Button(description="Search")
hbox = widgets.HBox([dropdown])
vbox = widgets.VBox([hbox, button])

# 顯示下拉選單和按鈕
display(vbox)

output = widgets.Output()
display(output)


# 定義按鈕事件處理函數
def on_button_clicked(b):
    with output:
        clear_output()
        print(f"正在搜尋: {dropdown.value}")
        driver = start_driver()

        if get_searchreview(driver, dropdown.value) == True:
            print(f"{dropdown.value} 搜尋成功")
            sleep(2)  # 等網頁加載
            page = 0

            scroll(driver, 10)

            while True:
                Listdata = TryGetRestaurantList(driver)
                saveJson("restaurant", Listdata, dropdown.value)
                sleep(3)
                break

        driver.quit()


# 綁定按鈕事件
button.on_click(on_button_clicked)

Output()